# DFL Basic Simulation - Results Analysis

This notebook provides analysis tools for DFL experiment results.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src to path
sys.path.append('../src')

from utils import MetricsTracker, Visualizer

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

%matplotlib inline

## Load Experiment Results

In [ ]:
# Load metrics from CSV files
experiment_name = "dfl_basic_cifar10"  # Change this to your experiment name

# Load node metrics
metrics_file = f"../results/{experiment_name}_metrics.csv"
if os.path.exists(metrics_file):
    metrics_df = pd.read_csv(metrics_file)
    print(f"Loaded node metrics: {metrics_df.shape}")
    print(metrics_df.head())
else:
    print(f"Metrics file not found: {metrics_file}")
    metrics_df = None

# Load global metrics
global_metrics_file = f"../results/{experiment_name}_global_metrics.csv"
if os.path.exists(global_metrics_file):
    global_metrics_df = pd.read_csv(global_metrics_file)
    print(f"\nLoaded global metrics: {global_metrics_df.shape}")
    print(global_metrics_df.head())
else:
    print(f"Global metrics file not found: {global_metrics_file}")
    global_metrics_df = None

## Basic Statistics

In [ ]:
if metrics_df is not None:
    print("=== Experiment Overview ===")
    print(f"Total rounds: {metrics_df['round'].max()}")
    print(f"Number of nodes: {metrics_df['node_id'].nunique()}")
    print(f"Total records: {len(metrics_df)}")
    
    print("\n=== Final Round Statistics ===")
    final_round = metrics_df[metrics_df['round'] == metrics_df['round'].max()]
    
    print(f"Final Training Accuracy: {final_round['train_accuracy'].mean():.4f} ± {final_round['train_accuracy'].std():.4f}")
    print(f"Final Test Accuracy: {final_round['test_accuracy'].mean():.4f} ± {final_round['test_accuracy'].std():.4f}")
    print(f"Final Training Loss: {final_round['train_loss'].mean():.4f} ± {final_round['train_loss'].std():.4f}")
    print(f"Final Test Loss: {final_round['test_loss'].mean():.4f} ± {final_round['test_loss'].std():.4f}")
    
    print("\n=== Data Distribution ===")
    data_sizes = final_round.set_index('node_id')['data_size'].to_dict()
    for node_id, size in data_sizes.items():
        print(f"Node {node_id}: {size} samples ({size/sum(data_sizes.values())*100:.1f}%)")

## Convergence Analysis

In [ ]:
if global_metrics_df is not None:
    # Create convergence plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    rounds = global_metrics_df['round']
    
    # Training accuracy
    if 'global_train_accuracy' in global_metrics_df.columns:
        ax1.plot(rounds, global_metrics_df['global_train_accuracy'], 'b-', linewidth=2, label='Global')
        if 'std_train_accuracy' in global_metrics_df.columns:
            ax1.fill_between(rounds, 
                           global_metrics_df['global_train_accuracy'] - global_metrics_df['std_train_accuracy'],
                           global_metrics_df['global_train_accuracy'] + global_metrics_df['std_train_accuracy'],
                           alpha=0.3)
    ax1.set_title('Training Accuracy Convergence')
    ax1.set_xlabel('Round')
    ax1.set_ylabel('Accuracy')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Test accuracy
    if 'global_test_accuracy' in global_metrics_df.columns:
        ax2.plot(rounds, global_metrics_df['global_test_accuracy'], 'r-', linewidth=2, label='Global')
        if 'std_test_accuracy' in global_metrics_df.columns:
            ax2.fill_between(rounds, 
                           global_metrics_df['global_test_accuracy'] - global_metrics_df['std_test_accuracy'],
                           global_metrics_df['global_test_accuracy'] + global_metrics_df['std_test_accuracy'],
                           alpha=0.3)
    ax2.set_title('Test Accuracy Convergence')
    ax2.set_xlabel('Round')
    ax2.set_ylabel('Accuracy')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # Training loss
    if 'global_train_loss' in global_metrics_df.columns:
        ax3.plot(rounds, global_metrics_df['global_train_loss'], 'g-', linewidth=2, label='Global')
        if 'std_train_loss' in global_metrics_df.columns:
            ax3.fill_between(rounds, 
                           global_metrics_df['global_train_loss'] - global_metrics_df['std_train_loss'],
                           global_metrics_df['global_train_loss'] + global_metrics_df['std_train_loss'],
                           alpha=0.3)
    ax3.set_title('Training Loss Convergence')
    ax3.set_xlabel('Round')
    ax3.set_ylabel('Loss')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # Test loss
    if 'global_test_loss' in global_metrics_df.columns:
        ax4.plot(rounds, global_metrics_df['global_test_loss'], 'm-', linewidth=2, label='Global')
        if 'std_test_loss' in global_metrics_df.columns:
            ax4.fill_between(rounds, 
                           global_metrics_df['global_test_loss'] - global_metrics_df['std_test_loss'],
                           global_metrics_df['global_test_loss'] + global_metrics_df['std_test_loss'],
                           alpha=0.3)
    ax4.set_title('Test Loss Convergence')
    ax4.set_xlabel('Round')
    ax4.set_ylabel('Loss')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    plt.tight_layout()
    plt.show()

## Node-wise Analysis

In [ ]:
if metrics_df is not None:
    # Plot individual node performance
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Training accuracy by node
    for node_id in metrics_df['node_id'].unique():
        node_data = metrics_df[metrics_df['node_id'] == node_id]
        ax1.plot(node_data['round'], node_data['train_accuracy'], 
                label=f'Node {node_id}', alpha=0.7)
    ax1.set_title('Training Accuracy by Node')
    ax1.set_xlabel('Round')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Test accuracy by node
    for node_id in metrics_df['node_id'].unique():
        node_data = metrics_df[metrics_df['node_id'] == node_id]
        ax2.plot(node_data['round'], node_data['test_accuracy'], 
                label=f'Node {node_id}', alpha=0.7)
    ax2.set_title('Test Accuracy by Node')
    ax2.set_xlabel('Round')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Training loss by node
    for node_id in metrics_df['node_id'].unique():
        node_data = metrics_df[metrics_df['node_id'] == node_id]
        ax3.plot(node_data['round'], node_data['train_loss'], 
                label=f'Node {node_id}', alpha=0.7)
    ax3.set_title('Training Loss by Node')
    ax3.set_xlabel('Round')
    ax3.set_ylabel('Loss')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Test loss by node
    for node_id in metrics_df['node_id'].unique():
        node_data = metrics_df[metrics_df['node_id'] == node_id]
        ax4.plot(node_data['round'], node_data['test_loss'], 
                label=f'Node {node_id}', alpha=0.7)
    ax4.set_title('Test Loss by Node')
    ax4.set_xlabel('Round')
    ax4.set_ylabel('Loss')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Data Distribution Analysis

In [ ]:
if metrics_df is not None:
    # Analyze data distribution across nodes
    final_round = metrics_df[metrics_df['round'] == metrics_df['round'].max()]
    data_sizes = final_round.set_index('node_id')['data_size'].to_dict()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar plot of data sizes
    nodes = list(data_sizes.keys())
    sizes = list(data_sizes.values())
    
    bars = ax1.bar(nodes, sizes)
    ax1.set_xlabel('Node ID')
    ax1.set_ylabel('Dataset Size')
    ax1.set_title('Data Distribution Across Nodes')
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom')
    
    # Pie chart
    ax2.pie(sizes, labels=[f'Node {i}' for i in nodes], autopct='%1.1f%%')
    ax2.set_title('Data Distribution (Percentage)')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate heterogeneity metrics
    sizes_array = np.array(sizes)
    mean_size = np.mean(sizes_array)
    std_size = np.std(sizes_array)
    cv = std_size / mean_size if mean_size > 0 else 0
    
    print(f"\n=== Data Heterogeneity Analysis ===")
    print(f"Mean dataset size: {mean_size:.1f}")
    print(f"Standard deviation: {std_size:.1f}")
    print(f"Coefficient of variation: {cv:.3f}")
    print(f"Min/Max ratio: {min(sizes)/max(sizes):.3f}")

## Performance Comparison

In [ ]:
if metrics_df is not None:
    # Compare final performance across nodes
    final_round = metrics_df[metrics_df['round'] == metrics_df['round'].max()]
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Test accuracy distribution
    ax1.bar(final_round['node_id'], final_round['test_accuracy'])
    ax1.set_xlabel('Node ID')
    ax1.set_ylabel('Test Accuracy')
    ax1.set_title('Final Test Accuracy by Node')
    ax1.grid(True, alpha=0.3)
    
    # Test loss distribution
    ax2.bar(final_round['node_id'], final_round['test_loss'], color='orange')
    ax2.set_xlabel('Node ID')
    ax2.set_ylabel('Test Loss')
    ax2.set_title('Final Test Loss by Node')
    ax2.grid(True, alpha=0.3)
    
    # Accuracy vs Data Size
    ax3.scatter(final_round['data_size'], final_round['test_accuracy'], s=100, alpha=0.7)
    ax3.set_xlabel('Dataset Size')
    ax3.set_ylabel('Test Accuracy')
    ax3.set_title('Test Accuracy vs Dataset Size')
    ax3.grid(True, alpha=0.3)
    
    # Add correlation coefficient
    corr = final_round[['data_size', 'test_accuracy']].corr().iloc[0, 1]
    ax3.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
            transform=ax3.transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Loss vs Data Size
    ax4.scatter(final_round['data_size'], final_round['test_loss'], s=100, alpha=0.7, color='orange')
    ax4.set_xlabel('Dataset Size')
    ax4.set_ylabel('Test Loss')
    ax4.set_title('Test Loss vs Dataset Size')
    ax4.grid(True, alpha=0.3)
    
    # Add correlation coefficient
    corr = final_round[['data_size', 'test_loss']].corr().iloc[0, 1]
    ax4.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
            transform=ax4.transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

## Export Results

In [ ]:
# Save analysis results
if metrics_df is not None and global_metrics_df is not None:
    print("Saving analysis results...")
    
    # Create summary statistics
    final_round = metrics_df[metrics_df['round'] == metrics_df['round'].max()]
    
    summary = {
        'experiment_name': experiment_name,
        'total_rounds': int(metrics_df['round'].max()),
        'num_nodes': int(metrics_df['node_id'].nunique()),
        'final_global_test_accuracy': float(global_metrics_df['global_test_accuracy'].iloc[-1]),
        'final_global_test_loss': float(global_metrics_df['global_test_loss'].iloc[-1]),
        'best_global_test_accuracy': float(global_metrics_df['global_test_accuracy'].max()),
        'best_round': int(global_metrics_df.loc[global_metrics_df['global_test_accuracy'].idxmax(), 'round']),
        'accuracy_std_final': float(final_round['test_accuracy'].std()),
        'loss_std_final': float(final_round['test_loss'].std())
    }
    
    # Save summary
    summary_df = pd.DataFrame([summary])
    summary_file = f"../results/{experiment_name}_analysis_summary.csv"
    summary_df.to_csv(summary_file, index=False)
    
    print(f"Analysis summary saved to: {summary_file}")
    print("\n=== Final Summary ===")
    for key, value in summary.items():
        print(f"{key}: {value}")